<a href="https://colab.research.google.com/github/Tom-Lennox/Kaggle/blob/master/titanic_20200410_%E3%81%AE%E3%82%B3%E3%83%94%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# title

In [2]:
# kaggle APIセット
!pip install kaggle

from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)
# 「Download 100%.」と表示で成功。

# [kaggle.json]を持参する。
from google.colab import drive
drive.mount('/content/drive')

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c titanic

# jsonファイルを指定の場所に配置
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
# titanicをダウンロードします。
!kaggle competitions download -c titanic
!ls

Download 100%.
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
cp: cannot create regular file '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, s

In [21]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
# RandomForestClassifier
# 複数の決定木を使って各決定木の予測結果の多数決で結果を求める方法
# ｜Decision Tree(決定木
# 　｜Classification Tree 分類木
# 　｜Regression Tree 回帰木

train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")
# とりあえずファイルを読み込む

# train.head()
# 名前確かに敬称あるね。
# ticketはアルファベットがヒントになりそう。
# Name: 敬称を別カラムに移植、削除
# objectで抽出したものをone-hot-encordingで置換（手動置換は無駄。
# ticket ⇒ 条件でカラム作成して代入 ⇒ 

print('▼　inull list')
print(train.isnull().sum()[train.isnull().sum() > 0].index.tolist())
# null抽出

print('▼　Embarked fillna')
# Embarked　処理：
# １．noneで置き換えてohe
# ２．mean()
train.head()
train['Embarked'] = train['Embarked'].fillna("None")
print(train['Embarked'].unique())

print('▼ Ticket')
train.head()
print(train['Ticket'].unique().tolist())

print('▼ Cabin')
train['Cabin'] = train['Cabin'].fillna("None")
print(train['Cabin'].isnull().sum())
# 【統合先】

▼　inull list
['Age', 'Cabin', 'Embarked']
▼　Embarked fillna
['S' 'C' 'Q' 'None']
▼ Ticket
['A/5 21171', 'PC 17599', 'STON/O2. 3101282', '113803', '373450', '330877', '17463', '349909', '347742', '237736', 'PP 9549', '113783', 'A/5. 2151', '347082', '350406', '248706', '382652', '244373', '345763', '2649', '239865', '248698', '330923', '113788', '347077', '2631', '19950', '330959', '349216', 'PC 17601', 'PC 17569', '335677', 'C.A. 24579', 'PC 17604', '113789', '2677', 'A./5. 2152', '345764', '2651', '7546', '11668', '349253', 'SC/Paris 2123', '330958', 'S.C./A.4. 23567', '370371', '14311', '2662', '349237', '3101295', 'A/4. 39886', 'PC 17572', '2926', '113509', '19947', 'C.A. 31026', '2697', 'C.A. 34651', 'CA 2144', '2669', '113572', '36973', '347088', 'PC 17605', '2661', 'C.A. 29395', 'S.P. 3464', '3101281', '315151', 'C.A. 33111', 'S.O.C. 14879', '2680', '1601', '348123', '349208', '374746', '248738', '364516', '345767', '345779', '330932', '113059', 'SO/C 14885', '3101278', 'W./C. 66

In [22]:
# ▼ all_data作成
train = pd.read_csv("/content/train.csv")
test_x = pd.read_csv("/content/test.csv")

train_x = train.drop("Survived",axis=1)
train_y = train["Survived"]
test_x = pd.read_csv("/content/test.csv")

all_data = pd.concat([train_x,test_x],axis=0,sort=False)
#学習データとテストデータを統合、カラムsortされると困る。

print('▼ 欠損　確認')
print(all_data.isnull().sum()[all_data.isnull().sum() > 0].index.tolist())

print('▼ 欠損　個数')
print(all_data.isnull().sum()[all_data.isnull().sum() > 0])

print('▼ 欠損　data')
print(all_data['Fare'][all_data['Fare'].isnull() == True])

print('▼ 補完')
all_data['Age'].fillna(all_data.Age.mean(), inplace=True)
all_data['Embarked'] = all_data['Embarked'].fillna("None")
all_data['Cabin'] = all_data['Cabin'].fillna("None")
all_data['Fare'].fillna(all_data.Fare.mean(), inplace=True)
# object, intに分けて自動処理。

print('▼ 補完　確認')
print(all_data.isnull().sum()[all_data.isnull().sum() > 0])

# train['Age'].fillna(train.Age.mean(), inplace=True)
all_data[['Age', 'Cabin', 'Embarked', 'Fare']]
# ▲ all_data作成

▼ 欠損　確認
['Age', 'Fare', 'Cabin', 'Embarked']
▼ 欠損　個数
Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64
▼ 欠損　data
152   NaN
Name: Fare, dtype: float64
▼ 補完
▼ 補完　確認
Series([], dtype: int64)


,Age,Cabin,Embarked,Fare
0,22.000000,None,S,7.2500
1,38.000000,C85,C,71.2833
2,26.000000,None,S,7.9250
3,35.000000,C123,S,53.1000
4,35.000000,None,S,8.0500
...,...,...,...,...
413,29.881138,None,S,8.0500
414,39.000000,C105,C,108.9000
415,38.500000,None,S,7.2500
416,29.881138,None,S,8.0500


In [23]:
combine1 = [all_data]

for all_data in combine1: 
  all_data['Salutation'] = all_data.Name.str.extract(' ([A-Za-z]+).', expand=False) 
for all_data in combine1: 
  all_data['Salutation'] = all_data['Salutation'].replace(['Lady', 'Countess','Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
  all_data['Salutation'] = all_data['Salutation'].replace('Mlle', 'Miss')
  all_data['Salutation'] = all_data['Salutation'].replace('Ms', 'Miss')
  all_data['Salutation'] = all_data['Salutation'].replace('Mme', 'Mrs')
Salutation_mapping ={"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5} 
for tain in combine1:
  all_data['Salutation'] = all_data['Salutation'].map(Salutation_mapping)
  all_data['Salutation'] = all_data['Salutation'].fillna(0)
  print(all_data['Salutation'].unique())
  del all_data['Salutation']
  del all_data['Name']

# ▲
# ▼ Ticket
for all_data in combine1:
  all_data['ticket_left'] = all_data['Ticket'].apply(lambda x: str(x)[0])
  # strに変換 + 1文字目を取得
  all_data['ticket_left'] = all_data['ticket_left'].apply(lambda x: str(x))
  all_data['del_ticket_left_zero'] = np.where((all_data['ticket_left']).isin(['W', '4', '7', '6', 'L', '5', '8']), '0','0')
  all_data['ticket_left'] = np.where((all_data['ticket_left']).isin(['1', '2', '3', 'S', 'P', 'C', 'A']), all_data['ticket_left'], all_data['del_ticket_left_zero'])
  del all_data['del_ticket_left_zero']
  all_data['tcker_len'] = all_data['Ticket'].apply(lambda x: len(x))
  del all_data['Ticket']
all_data['ticket_left']=all_data['ticket_left'].replace("1",1).replace("2",2).replace("3",3).replace("0",0).replace("S",3).replace("P",0).replace("C",3).replace("A",3)
print('▼ print(all_data["ticket_left"])')
print(all_data['ticket_left'].unique())
# ▲ Ticket
# ▼ Cabin
for all_data in combine1:
  all_data['cabin_left'] = all_data['Cabin'].apply(lambda x: str(x)[0])
  all_data['cabin_left'] = all_data['cabin_left'].apply(lambda x: str(x))
  # print(all_data['cabin_left'].unique())
  # ▲ コピペ用
  all_data['del_cabin_left_zero'] = np.where((all_data['cabin_left']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']), '0','0')
  # ▲ ！手直しの必要有。
  all_data['cabin_left'] = np.where((all_data['cabin_left']).isin([ 'F', 'E', 'D', 'C', 'B', 'A']), all_data['cabin_left'], all_data['del_cabin_left_zero'])
  # del all_data['Cabin']
  del all_data['del_cabin_left_zero']
  del all_data['Cabin']
all_data['cabin_left']=all_data['cabin_left'].replace("A",1).replace("B",2).replace("C",1).replace("0",0).replace("D",2).replace("E",2).replace("F",1)
# ▲ ！手直しの必要有。
# ▲ Cabin
# ▼ family_of　（新規追加）
all_data['family_of'] = all_data['SibSp'] + all_data['Parch'] + 1
for all_data in combine1:
  all_data['is_alone'] = 0
  all_data.loc[all_data['family_of'] == 1, 'is_alone'] = 1
# ▲ family_of　（新規追加）

[1. 3. 2. 4. 0. 5.]
▼ print(all_data["ticket_left"])
[3 0 1 2]


In [24]:
# print(all_data['Embarked'].unique())
# all_data.info()
# print(all_data.dtypes[all_data.dtypes=="object"].index.tolist())
# cal_list = all_data.dtypes[all_data.dtypes=="object"].index.tolist()
# print(cal_list)
# all_data.info()
# test_x.info()

all_data.info()
all_data

Embark_dum  = pd.get_dummies(all_data['Embarked'])
Embark_dum.columns = ['Class1','Class2','Class3', 'Class4']
Sex_dum = pd.get_dummies(all_data['Sex'])
Sex_dum.columns = ['Sex1','Sex2']
all_data = all_data.join(Embark_dum)
all_data = all_data.join(Sex_dum)

del all_data['Embarked']
del all_data['Sex']

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Pclass       1309 non-null   int64  
 2   Sex          1309 non-null   object 
 3   Age          1309 non-null   float64
 4   SibSp        1309 non-null   int64  
 5   Parch        1309 non-null   int64  
 6   Fare         1309 non-null   float64
 7   Embarked     1309 non-null   object 
 8   ticket_left  1309 non-null   int64  
 9   tcker_len    1309 non-null   int64  
 10  cabin_left   1309 non-null   int64  
 11  family_of    1309 non-null   int64  
 12  is_alone     1309 non-null   int64  
dtypes: float64(2), int64(9), object(2)
memory usage: 143.2+ KB


In [27]:
test_x

,PassengerId,Pclass,Age,SibSp,Parch,Fare,ticket_left,tcker_len,cabin_left,family_of,is_alone,Class1,Class2,Class3,Class4,Sex1,Sex2
0,112,3,14.500000,1,0,14.4542,2,4,0,2,0,0,0,1,0,1,0
1,1003,3,29.881138,0,0,7.7792,3,6,0,1,1,1,0,0,0,1,0
2,1003,3,29.881138,0,0,7.7792,3,6,0,1,1,1,0,0,0,1,0
3,1003,3,29.881138,0,0,7.7792,3,6,0,1,1,0,0,1,0,1,0
4,1003,3,29.881138,0,0,7.7792,3,6,0,1,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,887,2,27.000000,0,0,13.0000,2,6,0,1,1,0,0,0,1,0,1
2922,888,1,19.000000,0,0,30.0000,1,6,2,1,1,0,0,0,1,1,0
2923,889,3,29.881138,1,2,23.4500,0,10,0,4,0,0,0,0,1,1,0
2924,890,1,26.000000,0,0,30.0000,1,6,1,1,1,1,0,0,0,0,1


In [25]:
# ▼ train
#カテゴリ変数となっているカラムを取り出す
cal_list = all_data.dtypes[all_data.dtypes=="object"].index.tolist()

#カテゴリ変数をget_dummiesによるone-hot-encodingを行う
# all_data = pd.get_dummies(all_data,columns=cal_list)

train_x = all_data.iloc[:train_x.shape[0],:].reset_index(drop=True)
test_x = all_data.iloc[train_x.shape[0]:,:].reset_index(drop=True)
# 分割

print("train_x: "+str(train_x.shape))
print("test_x: "+str(test_x.shape))
#サイズを確認

print(train_x.isnull().sum()[train_x.isnull().sum() > 0])
print(test_x.isnull().sum()[test_x.isnull().sum() > 0])

train_data = train_x.values
xs = train_data[:, 2:]
# Pclass以降の変数
y  = train_y
# 正解データ

test_data = test_x.values
xs_test = test_data[:, 1:]
# ▲ 

train_x: (891, 17)
test_x: (2926, 17)
Series([], dtype: int64)
Series([], dtype: int64)


In [26]:
# all_data.info()
# all_data
# xs
print('▼ train')
train_x.info()
print('▼ test')
test_x.info()

▼ train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Age          891 non-null    float64
 3   SibSp        891 non-null    int64  
 4   Parch        891 non-null    int64  
 5   Fare         891 non-null    float64
 6   ticket_left  891 non-null    int64  
 7   tcker_len    891 non-null    int64  
 8   cabin_left   891 non-null    int64  
 9   family_of    891 non-null    int64  
 10  is_alone     891 non-null    int64  
 11  Class1       891 non-null    uint8  
 12  Class2       891 non-null    uint8  
 13  Class3       891 non-null    uint8  
 14  Class4       891 non-null    uint8  
 15  Sex1         891 non-null    uint8  
 16  Sex2         891 non-null    uint8  
dtypes: float64(2), int64(9), uint8(6)
memory usage: 81.9 KB
▼ test
<class 'pandas.core.f

In [28]:
from sklearn.ensemble import RandomForestClassifier

random_forest=RandomForestClassifier()
random_forest.fit(xs, y)
Y_pred = random_forest.predict(xs_test)

# import csv
# with open("predict_result_data.csv", "w") as f:
#     writer = csv.writer(f, lineterminator='\n')
#     writer.writerow(["PassengerId", "Survived"])
#     for pid, survived in zip(test_data[:,0].astype(int), Y_pred.astype(int)):
#         writer.writerow([pid, survived])

ValueError: ignored

In [0]:
# ▽ 以下、
train_data = train.values
xs = train_data[:, 2:] # Pclass以降の変数
y  = train_data[:, 1]  # 正解データ
# △ 

In [0]:


# ▼ohe関連
# # 追記
# # ▼
# # 処理のために結合
# train_x = pd.read_csv("/content/train.csv")
# test_x = pd.read_csv("/content/test.csv")
# print('train_x: ' + str(train_x.shape))
# print('test_x: '  + str(test_x.shape))
# # sort: カラム名のsort。
# all_data = pd.concat([train_x,test_x],axis=0,sort=True)
# print('all_data: '  + str(all_data.shape))
# # ▲
# # ▼
# # # 学習データとテストデータに再分割
# # [tips]shape使え。
# # train_x = all_data.iloc[:train_x.shape[0],:].reset_index(drop=True)
# # test_x = all_data.iloc[train_x.shape[0]:,:].reset_index(drop=True)
# # ▲
# # ▼
# # nullは下記で取得できる。
# print('▼train_x')
# print(train_x.isnull().sum())
# print('▼test_x')
# print(test_x.isnull().sum())
# print(train[['Age','Cabin','Embarked']])
# all_data.head(3)
# # ▲
# # ▼
# # pandas, one-hot-encodingを行う。
# one-hot-encoding
# cal_list = all_data.dtypes[all_data.dtypes=="object"].index.tolist()
# all_data = pd.get_dummies(all_data,columns=cal_list)
# all_data
# # ▲